In [22]:
import torch.optim as optim
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision.datasets as datasets
import torch
import torch.nn as nn
from torch.utils.data import DataLoader


import src.utils as u
from src.models import ResNet
from src.quantize import quantize
from tqdm.notebook import tqdm

In [23]:
def _weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            if option == 'A':
                """
                For CIFAR10 ResNet paper uses option A.
                """
                self.shortcut = LambdaLayer(lambda x:
                                            F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                     nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                     nn.BatchNorm2d(self.expansion * planes)
                )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64, num_classes)

        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def resnet20():
    return ResNet(BasicBlock, [3, 3, 3])

def get_model2(model, learning_rate=1e-3, weight_decay=1e-4):

    # set the first layer not trainable
    # model.features.conv0.weight.requires_grad = False

    # all fc layers
    weights = [
        p for n, p in model.named_parameters()
        if 'weight' in n and 'conv' not in n
    ]

    # all conv layers
    weights_to_be_quantized = [
        p for n, p in model.named_parameters()
        # if 'conv' in n and 'conv0' not in n
        if 'conv' in n and 'weight' in n
    ]

    biases = [
        p for n, p in model.named_parameters()
        if 'bias' in n
    ]    

    params = [
        {'params': weights, 'weight_decay': weight_decay},
        {'params': weights_to_be_quantized, 'weight_decay': weight_decay},
        {'params': biases,  'weight_decay': weight_decay}
    ]
    optimizer = optim.SGD(params, lr=learning_rate, momentum=0.9)

    loss = nn.CrossEntropyLoss().cuda()
    model = model.cuda()  # move the model to gpu
    return model, loss, optimizer
def quantize_bw(kernel : torch.Tensor):
    """
    binary quantization
    Return quantized weights of a layer.
    """
    delta = kernel.abs().mean()
    sign = kernel.sign().float()



    return sign*delta

In [24]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
quantize_nbits = 8
num_epochs = 200
batch_size_test = 100
batch_size_train = 128
learning_rate = 0.001
eta_rate = 1.05
eta = 1
stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        # (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
        # ((0.507, 0.487, 0.441), (0.267, 0.256, 0.276))
global best_acc
best_acc = 0
best_count = 0

In [25]:
# Data augmentation and normalization for training
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(*stats)
])

# Normalization for testing
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(*stats)
])

# CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform_train, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform_test)

batch_size_test = int(len(test_dataset)/40 )#100
# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size_train, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size_test, shuffle=False)
#--------------------------------------------------------------------------
    

Files already downloaded and verified


In [26]:
net = resnet20()
net.to(device)


ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

In [27]:
net, criterion, optimizer = get_model2(net, learning_rate=0.1, weight_decay=5e-4)

    
all_G_kernels = [
    Variable(kernel.data.clone(), requires_grad=True)
    for kernel in optimizer.param_groups[1]['params']
]


all_W_kernels = [kernel for kernel in optimizer.param_groups[1]['params']]
kernels = [{'params': all_G_kernels}]
optimizer_quant = optim.SGD(kernels, lr=0)

scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[80,120,160], gamma=0.1)

In [28]:

#for epoch in tqdm(range(num_epochs)):    num_epochs = 200
for epoch in (pbar := tqdm(range(num_epochs))):
    #print('Epoch ID', epoch)
    #----------------------------------------------------------------------
    # Training
    #----------------------------------------------------------------------
    pbar.set_description(f"Training {epoch}",refresh=True)
    correct = 0; total = 0; train_loss = 0
    net.train()
    for batch_idx, (x, target) in enumerate(tqdm(train_loader, leave=False)):
    #for batch_idx, (x, target) in enumerate(train_loader):
        #if batch_idx < 1:
        optimizer.zero_grad()
        x, target = Variable(x.cuda()), Variable(target.cuda())
        all_W_kernels = optimizer.param_groups[1]['params']
        all_G_kernels = optimizer_quant.param_groups[0]['params']
        
        for i in range(len(all_W_kernels)):
            k_W = all_W_kernels[i]
            k_G = all_G_kernels[i]
            V = k_W.data
            #print(type(V))
            #####Binary Connect#########################
            #k_G.data = quantize_bw(V)
            ############################################
            
            ######Binary Relax##########################
            if epoch<120:
                #k_G.data = (eta*quantize_bw(V)+V)/(1+eta)
                k_G.data = (eta*quantize(V,num_bits=quantize_nbits)+V)/(1+eta)
                
            else:
                k_G.data = quantize(V, num_bits=quantize_nbits)
            #############################################
            
            k_W.data, k_G.data = k_G.data, k_W.data
            
            
        score = net(x)
        loss = criterion(score, target)
        loss.backward()
        
        for i in range(len(all_W_kernels)):
            k_W = all_W_kernels[i]
            k_G = all_G_kernels[i]
            k_W.data, k_G.data = k_G.data, k_W.data
        
        optimizer.step()
        
        train_loss += loss.data
        _, predicted = torch.max(score.data, 1)
        total += target.size(0)
        correct += predicted.eq(target.data).cpu().sum()
        pbar.write(f"Training {batch_idx} Loss: {train_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f}% ({correct}/{total})")
        
    #----------------------------------------------------------------------
    # Testing
    #----------------------------------------------------------------------
    pbar.set_description(f"Testing {epoch}",refresh=True)
    test_loss = 0; correct = 0; total = 0
    net.eval()
    
    for i in range(len(all_W_kernels)):
        k_W = all_W_kernels[i]
        k_quant = all_G_kernels[i]    
        k_W.data, k_quant.data = k_quant.data, k_W.data
    with torch.no_grad():
        for batch_idx, (x, target) in enumerate(tqdm(test_loader, leave=False)):
        #for batch_idx, (x, target) in enumerate(test_loader):
            x, target = Variable(x.cuda()), Variable(target.cuda())
            score= net(x)
            
            loss = criterion(score, target)
            test_loss += loss.data
            _, predicted = torch.max(score.data, 1)
            total += target.size(0)
            correct += predicted.eq(target.data).cpu().sum()
            pbar.write(f"Testing {batch_idx} Loss: {train_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f}% ({correct}/{total})")

    
    #----------------------------------------------------------------------
    # Save the checkpoint
    #----------------------------------------------------------------------
    '''
    for i in range(len(all_W_kernels)):
        k_W = all_W_kernels[i]
        k_quant = all_G_kernels[i]    
        k_W.data, k_quant.data = k_quant.data, k_W.data
        '''  
    acc = 100.*correct/total
    #if acc > best_acc:
    if correct > best_count:
        print('Saving model...')
        state = {
            'state': net.state_dict(), #net,
            'acc': acc,
            'epoch': epoch,
        }
        
        torch.save(state, f'./models/resnet_{quantize_nbits}bits.pth')
        #net.save_state_dict('resnet20.pt')
        best_acc = acc
        best_count = correct

    for i in range(len(all_W_kernels)):
        k_W=all_W_kernels[i]
        k_quant=all_W_kernels[i]
        k_W.data, k_quant.data =k_quant.data,k_W.data


  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]

0 391 Loss: 4.362 | Acc: 10.156% (13/128)
1 391 Loss: 3.798 | Acc: 9.375% (24/256)
2 391 Loss: 3.608 | Acc: 12.760% (49/384)
3 391 Loss: 3.909 | Acc: 11.914% (61/512)
4 391 Loss: 3.929 | Acc: 12.344% (79/640)
5 391 Loss: 3.855 | Acc: 13.021% (100/768)
6 391 Loss: 3.818 | Acc: 14.062% (126/896)
7 391 Loss: 3.757 | Acc: 13.770% (141/1024)
8 391 Loss: 3.609 | Acc: 13.715% (158/1152)
9 391 Loss: 3.549 | Acc: 13.594% (174/1280)
10 391 Loss: 3.477 | Acc: 13.565% (191/1408)
11 391 Loss: 3.451 | Acc: 13.411% (206/1536)
12 391 Loss: 3.426 | Acc: 13.101% (218/1664)
13 391 Loss: 3.401 | Acc: 12.667% (227/1792)
14 391 Loss: 3.342 | Acc: 12.448% (239/1920)
15 391 Loss: 3.306 | Acc: 12.549% (257/2048)
16 391 Loss: 3.252 | Acc: 13.006% (283/2176)
17 391 Loss: 3.203 | Acc: 12.804% (295/2304)
18 391 Loss: 3.163 | Acc: 12.993% (316/2432)
19 391 Loss: 3.118 | Acc: 13.398% (343/2560)
20 391 Loss: 3.094 | Acc: 13.951% (375/2688)
21 391 Loss: 3.050 | Acc: 14.347% (404/2816)
22 391 Loss: 3.024 | Acc: 14.946%

  0%|          | 0/40 [00:00<?, ?it/s]

0 391 Loss: 704.478 | Acc: 49.200% (123/250)
1 391 Loss: 352.239 | Acc: 47.600% (238/500)
2 391 Loss: 234.826 | Acc: 45.733% (343/750)
3 391 Loss: 176.119 | Acc: 46.200% (462/1000)
4 391 Loss: 140.896 | Acc: 46.880% (586/1250)
5 391 Loss: 117.413 | Acc: 46.400% (696/1500)
6 391 Loss: 100.640 | Acc: 45.829% (802/1750)
7 391 Loss: 88.060 | Acc: 46.150% (923/2000)
8 391 Loss: 78.275 | Acc: 46.578% (1048/2250)
9 391 Loss: 70.448 | Acc: 46.480% (1162/2500)
10 391 Loss: 64.043 | Acc: 45.745% (1258/2750)
11 391 Loss: 58.706 | Acc: 45.733% (1372/3000)
12 391 Loss: 54.191 | Acc: 46.062% (1497/3250)
13 391 Loss: 50.320 | Acc: 46.143% (1615/3500)
14 391 Loss: 46.965 | Acc: 45.973% (1724/3750)
15 391 Loss: 44.030 | Acc: 46.100% (1844/4000)
16 391 Loss: 41.440 | Acc: 46.141% (1961/4250)
17 391 Loss: 39.138 | Acc: 46.222% (2080/4500)
18 391 Loss: 37.078 | Acc: 46.316% (2200/4750)
19 391 Loss: 35.224 | Acc: 46.360% (2318/5000)
20 391 Loss: 33.547 | Acc: 46.324% (2432/5250)
21 391 Loss: 32.022 | Acc: 

...

0 391 Loss: 0.471 | Acc: 85.156% (109/128)
1 391 Loss: 0.393 | Acc: 87.500% (224/256)
2 391 Loss: 0.404 | Acc: 87.500% (336/384)
3 391 Loss: 0.389 | Acc: 87.891% (450/512)
4 391 Loss: 0.392 | Acc: 87.188% (558/640)
5 391 Loss: 0.404 | Acc: 86.589% (665/768)
6 391 Loss: 0.395 | Acc: 87.054% (780/896)
7 391 Loss: 0.406 | Acc: 86.621% (887/1024)
8 391 Loss: 0.419 | Acc: 86.545% (997/1152)
9 391 Loss: 0.423 | Acc: 86.094% (1102/1280)
10 391 Loss: 0.427 | Acc: 85.795% (1208/1408)
11 391 Loss: 0.422 | Acc: 86.003% (1321/1536)
12 391 Loss: 0.425 | Acc: 85.697% (1426/1664)
13 391 Loss: 0.435 | Acc: 85.324% (1529/1792)
14 391 Loss: 0.434 | Acc: 85.469% (1641/1920)
15 391 Loss: 0.437 | Acc: 85.303% (1747/2048)
16 391 Loss: 0.436 | Acc: 84.972% (1849/2176)
17 391 Loss: 0.434 | Acc: 85.026% (1959/2304)
18 391 Loss: 0.434 | Acc: 85.074% (2069/2432)
19 391 Loss: 0.445 | Acc: 84.766% (2170/2560)
20 391 Loss: 0.445 | Acc: 84.859% (2281/2688)
21 391 Loss: 0.442 | Acc: 84.908% (2391/2816)
22 391 Loss: 0

  0%|          | 0/40 [00:00<?, ?it/s]

0 391 Loss: 175.692 | Acc: 84.000% (210/250)
1 391 Loss: 87.846 | Acc: 84.800% (424/500)
2 391 Loss: 58.564 | Acc: 85.200% (639/750)
3 391 Loss: 43.923 | Acc: 84.900% (849/1000)
4 391 Loss: 35.138 | Acc: 85.040% (1063/1250)
5 391 Loss: 29.282 | Acc: 84.867% (1273/1500)
6 391 Loss: 25.099 | Acc: 84.743% (1483/1750)
7 391 Loss: 21.961 | Acc: 84.450% (1689/2000)
8 391 Loss: 19.521 | Acc: 84.178% (1894/2250)
9 391 Loss: 17.569 | Acc: 83.960% (2099/2500)
10 391 Loss: 15.972 | Acc: 83.818% (2305/2750)
11 391 Loss: 14.641 | Acc: 83.767% (2513/3000)
12 391 Loss: 13.515 | Acc: 83.538% (2715/3250)
13 391 Loss: 12.549 | Acc: 83.600% (2926/3500)
14 391 Loss: 11.713 | Acc: 83.547% (3133/3750)
15 391 Loss: 10.981 | Acc: 83.650% (3346/4000)
16 391 Loss: 10.335 | Acc: 83.694% (3557/4250)
17 391 Loss: 9.761 | Acc: 83.911% (3776/4500)
18 391 Loss: 9.247 | Acc: 83.853% (3983/4750)
19 391 Loss: 8.785 | Acc: 83.760% (4188/5000)
20 391 Loss: 8.366 | Acc: 83.619% (4390/5250)
21 391 Loss: 7.986 | Acc: 83.545%

  0%|          | 0/391 [00:00<?, ?it/s]

0 391 Loss: 0.418 | Acc: 85.938% (110/128)
1 391 Loss: 0.374 | Acc: 87.500% (224/256)
2 391 Loss: 0.398 | Acc: 87.760% (337/384)
3 391 Loss: 0.444 | Acc: 85.352% (437/512)
4 391 Loss: 0.428 | Acc: 85.469% (547/640)
5 391 Loss: 0.434 | Acc: 85.417% (656/768)
6 391 Loss: 0.421 | Acc: 86.049% (771/896)
7 391 Loss: 0.439 | Acc: 85.254% (873/1024)
8 391 Loss: 0.430 | Acc: 85.590% (986/1152)
9 391 Loss: 0.424 | Acc: 85.781% (1098/1280)
10 391 Loss: 0.426 | Acc: 85.653% (1206/1408)
11 391 Loss: 0.434 | Acc: 85.221% (1309/1536)
12 391 Loss: 0.436 | Acc: 85.156% (1417/1664)
13 391 Loss: 0.431 | Acc: 85.268% (1528/1792)
14 391 Loss: 0.428 | Acc: 85.208% (1636/1920)
15 391 Loss: 0.434 | Acc: 85.010% (1741/2048)
16 391 Loss: 0.432 | Acc: 85.156% (1853/2176)
17 391 Loss: 0.429 | Acc: 85.243% (1964/2304)
18 391 Loss: 0.428 | Acc: 85.197% (2072/2432)
19 391 Loss: 0.425 | Acc: 85.312% (2184/2560)
20 391 Loss: 0.426 | Acc: 85.342% (2294/2688)
21 391 Loss: 0.429 | Acc: 85.263% (2401/2816)
22 391 Loss: 0